In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('Online Retail Data Set.csv', encoding='unicode_escape')
df = df.dropna()
df = df.drop_duplicates()

## we can convert customer ID float64 to int64
df.CustomerID = df.CustomerID.astype('Int64')
## Datetime to datetime
df.InvoiceDate = df.InvoiceDate.astype('datetime64[ns]')
df.info() # data types 

In [ ]:
#df.to_csv('new-data.csv')

In [2]:
df = pd.read_csv('new-data.csv', encoding='unicode_escape')

products = df['Description'].nunique()
print(products) # 3896

3896


In [ ]:
# how many uniqe stockcode there , are they unique at all? if yes then we can use them as primary key
stockcode = df['StockCode'].nunique()
print(stockcode) # 3684

# no, they aren't unique 

In [ ]:
CustomerID = df['CustomerID'].nunique()
print(CustomerID) # 4372
InvoiceNo = df['InvoiceNo'].nunique()
print(InvoiceNo) # 22190
Quantity = df['Quantity'].nunique()
print(Quantity) # 22190

UnitPrice = df['UnitPrice'].nunique()
print(UnitPrice) # 620

Country = df['Country'].nunique()
print(Country) # 37

'''
Column name: "InvoiceNo", datatype:int, total 22190 unique invoice no are there;
Column name: "StockCode", datatype: string, unique stockcode: 3684;
Column name: "Description", datatype: string, unique description: 3896;
Column name: "Quantity", datatype: int, unique Quantity: 22190;
Column name: "InvoiceDate", datatype: DATE Time, all rows are unique;
Column name: "UnitPrice", datatype: float, 620 unique unitprice;
Column name: "CustomerID", datatype: int, unique CustomerID: 4372;
Column name: "Country", datatype: string, unique Country: 37;
'''

In [ ]:
from sqlalchemy import create_engine

# Database connection settings
db_host = '127.0.0.1:3306'
db_user = 'root'
db_password = '1234'
db_name = 'online_retail_data_set'
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')
connector = engine.connect()

# query = 'SELECT * FROM online_retail_datat_set.customers;'
# result = connector.execute(text(query))



### Store unique CustomerID and Country Name into SQL DB

In [ ]:
# Create a dictionary to store unique customer IDs and their countries
customer_countries = {}
for index, row in df.iterrows():
    CustomerID = row['CustomerID']
    Country = row['Country']
    if CustomerID not in customer_countries:
        customer_countries[CustomerID] = Country

len(customer_countries)


##### i am sorting data to a csv file. with MySQL Workbench I will import data to datable direclty from .csv files 

In [21]:
df = pd.DataFrame(customer_countries.items(), columns=['CustomerID', 'Country'])
df.to_csv('customers.csv', index=False)

In [ ]:
# connector.close()

In [ ]:
from sqlalchemy import text
for customer_id, country in customer_countries.items():    
    customer_id = int(customer_id)
    country = str(country)
    print(f"Data inserted Customer ID: {customer_id}, Country: {country}")
    query = f'insert INTO online_retail_data_set.customers (CustomerID, Country) VALUES ({customer_id}, "{country}");'
    print(query)
    connector.execute(text(query))    

#### invoice - grouping invoice by invoiceno

In [11]:
invoice_grouped = df[['InvoiceNo', 'InvoiceDate','CustomerID']].groupby('InvoiceNo').first()
#top_prod_by_rev.style.background_gradient(text_color_threshold=.5)
invoice_grouped.to_csv('invoices.csv')

#### products

In [13]:
df_copy = df

products = df_copy[['ProductID', 'Description','UnitPrice']].groupby('Description').first()

products['ProductID'] = range(1000, 1000 + df['Description'].nunique()) # 3896 descriptions / products

products.to_csv('products.csv')

KeyError: "['ProductID'] not in index"